In [6]:
# Carica il file csv e coonverto colonna dei prezzi in int
import pandas as pd
import re
import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
import seaborn as sns
%matplotlib qt
from matplotlib.patches import Patch


df=pd.read_csv("milano_dati_booking.csv")
df['prezzo']= df['prezzo'].str.replace(".","")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df['prezzo_iniziale']= df['prezzo_iniziale'].str.replace(".","")
df['prezzo_iniziale'] = df['prezzo_iniziale'].apply(lambda x: int(re.search(r'\d+', x).group()))

In [7]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['offerta_solo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)

In [3]:
#Prendo tutte le combinazioni di coppie di ricerche sulla stessa città
df_list = df['TIMESTAMP'].unique()
coppie_df = list(itertools.combinations(df_list, 2))
len(coppie_df)


276

In [5]:
# Creo un Dataframe per vedere differenze di prezzo tra diverse ricerche
list_dict_df = []
for i in coppie_df:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    #merged_solo_nome = pd.merge(df1,df2, on='nome_hotel')
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"]).abs()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe_1"
    else: prezzi_inferiori = "Dataframe_2"
    try: 
        xor_booking_paga = len(merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])])
    except:
        xor_booking_paga = None
    try: 
        xor_offerta_solo_mobile = len(merged_df[(merged_df['offerta_solo_mobile_x']) ^ (merged_df['offerta_solo_mobile_y'])])
    except:
        xor_offerta_solo_mobile = None
    
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df['totale_corrispondenze'] = initial_len
    dict_df ['num_differenze_prezzo'] = len(merged_df)
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len*100
    #dict_df['hotel_scartati'] = len(merged_solo_nome) - initial_len
    dict_df['xor_booking_paga'] = xor_booking_paga
    dict_df['xor_offerta_solo_mobile'] = xor_offerta_solo_mobile
    dict_df['tipo_ricerche'] = df1['user_agent_type'].unique()[0] + "/"+ df2['user_agent_type'].unique()[0]
    dict_df['os'] = df1['os'].unique()[0] + "/"+ df2['os'].unique()[0]
    dict_df['username'] = df1['username'].unique()[0] + "/"+ df2['username'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    list_dict_df.append(dict_df)

df_diff_prezzo = pd.DataFrame(list_dict_df)   
pd.set_option('display.max_rows', None)
df_diff_prezzo.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

,coppia_ricerche,totale_corrispondenze,num_differenze_prezzo,media,mediana,%_prezzi_diversi,xor_booking_paga,xor_offerta_solo_mobile,tipo_ricerche,os,username,prezzi_inferiori
231,03-07-2024---15:20 *** 03-07-2024---15:32,954,937,0.0,0.0,98.218029,0,0,desktop/desktop,Linux/MacOS,pantilaura56/pantilaura56,Dataframe_2
48,03-07-2024---12:34 *** 03-07-2024---13:25,937,934,0.0,0.0,99.679829,41,0,desktop/desktop,Linux/MacOS,sunnytraveler/sunnytraveler,Dataframe_2
274,03-07-2024---17:21 *** 03-07-2024---17:51,928,926,0.0,0.0,99.784483,35,0,desktop/desktop,Windows/MacOS,marcofantile/marcofantile,Dataframe_2
156,03-07-2024---14:00 *** 03-07-2024---14:20,942,922,0.0,0.0,97.876858,23,0,mobile/mobile,Android/Android,pantilaura56/pantilaura56,Dataframe_2
126,03-07-2024---13:25 *** 03-07-2024---14:38,922,919,0.0,0.0,99.674620,39,0,desktop/desktop,MacOS/Windows,sunnytraveler/pantilaura56,Dataframe_2
23,03-07-2024---12:08 *** 03-07-2024---12:34,919,915,0.0,0.0,99.564744,38,0,desktop/desktop,Windows/Linux,sunnytraveler/sunnytraveler,Dataframe_2
123,03-07-2024---13:25 *** 03-07-2024---13:40,960,915,0.0,0.0,95.312500,0,0,desktop/desktop,MacOS/MacOS,sunnytraveler/sunnytraveler,Dataframe_2
130,03-07-2024---13:25 *** 03-07-2024---15:20,929,912,0.0,0.0,98.170075,41,0,desktop/desktop,MacOS/Linux,sunnytraveler/pantilaura56,Dataframe_2
202,03-07-2024---14:49 *** 03-07-2024---15:43,912,912,0.0,0.0,100.000000,0,0,desktop/desktop,Windows/MacOS,pantilaura56/pantilaura56,Dataframe_2
131,03-07-2024---13:25 *** 03-07-2024---15:32,911,908,0.0,0.0,99.670692,42,0,desktop/desktop,MacOS/MacOS,sunnytraveler/pantilaura56,Dataframe_2


In [5]:
# Trova le ricerche con maggior numero di sconti 
df_differenze_top = df_diff_prezzo[df_diff_prezzo['%_prezzi_diversi']>50]
df_differenze_top['ricerche_piu_presenti'] = df_differenze_top['coppia_ricerche'].str.replace('*** ','')
orari = df_differenze_top['ricerche_piu_presenti'].str.split().explode().value_counts()
orari

C:\Users\aless\AppData\Local\Temp\ipykernel_31412\217838661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_differenze_top['ricerche_piu_presenti'] = df_differenze_top['coppia_ricerche'].str.replace('*** ','')


ricerche_piu_presenti
03-07-2024---17:32    22
03-07-2024---16:09    22
03-07-2024---11:52     2
03-07-2024---14:49     2
03-07-2024---17:21     2
03-07-2024---17:03     2
03-07-2024---16:51     2
03-07-2024---15:55     2
03-07-2024---15:43     2
03-07-2024---15:32     2
03-07-2024---15:20     2
03-07-2024---15:08     2
03-07-2024---14:59     2
03-07-2024---14:38     2
03-07-2024---14:20     2
03-07-2024---14:00     2
03-07-2024---13:40     2
03-07-2024---13:25     2
03-07-2024---13:14     2
03-07-2024---13:02     2
03-07-2024---12:46     2
03-07-2024---12:34     2
03-07-2024---12:08     2
03-07-2024---17:51     2
Name: count, dtype: int64

In [6]:
# Visualizza un grafico in cui mostra la distribuzione delle differenze di prezzo in base alle ricerche
# creo anche 3 zone del grafico in base alle percentuali rilevanti di differenze di prezzo
df_diff_prezzo_presenti = df_diff_prezzo[df_diff_prezzo['%_prezzi_diversi']>0]
zone1 = df_diff_prezzo_presenti[df_diff_prezzo_presenti['%_prezzi_diversi']<20]
zone2 = df_diff_prezzo_presenti[(df_diff_prezzo_presenti['%_prezzi_diversi']<40)&(df_diff_prezzo_presenti['%_prezzi_diversi']>20)]
zone3 = df_diff_prezzo_presenti[df_diff_prezzo_presenti['%_prezzi_diversi']>60]
desk_desk = df_diff_prezzo_presenti[(df_diff_prezzo_presenti['tipo_ricerche']=="mobile/mobile")|(df_diff_prezzo_presenti['tipo_ricerche']=="desktop/desktop")]
desk_mob = df_diff_prezzo_presenti[(df_diff_prezzo_presenti['tipo_ricerche']=="mobile/desktop")|(df_diff_prezzo_presenti['tipo_ricerche']=="desktop/mobile")]
bins = 30
plt.style.use("seaborn-v0_8-dark")
plt.hist(desk_desk['num_differenze_prezzo'],bins = 20 , color='skyblue', alpha=0.7, label='Desktop-Desktop or Mobile-Mobile')
plt.hist(desk_mob['num_differenze_prezzo'], bins = 18 ,color='lightgreen', alpha=0.7, label='Desktop-Mobile')
#plt.hist(zone3['num_differenze_prezzo'], bins = 16 ,color='red', alpha=0.7, label='Ricerche di tutti i tipi')
plt.axvline(df_diff_prezzo_presenti['num_differenze_prezzo'].mean(),color="grey", linewidth="1",label ="Mean",alpha=0.3)
plt.text(35, 10,str(zone1['media'].mean().round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(222, 10,str(zone2['media'].mean().round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(560, 10,str(zone3['media'].mean().round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.legend(fontsize=14)
plt.title('Distribution of research pairs with price differences',fontsize=18)
plt.xlabel('Number of different prices between one search and another',fontsize=14)
#plt.ylabel('Ricerche',fontsize=14)
plt.yticks([])
plt.show()

In [7]:
# divide la zona 3 in base alle due ricerche principali, questo serve perchè c'è una grossa differenza tra le  due ricerche
zone3.sort_values(by=['xor_booking_paga','media'],ascending=False)
zone3r1 = zone3[zone3['coppia_ricerche'].str.contains("16:09")]
zone3r2 = zone3[zone3['coppia_ricerche'].str.contains("17:32")]
zone3r2[zone3r2['tipo_ricerche']!="mobile/mobile"]


,coppia_ricerche,totale_corrispondenze,num_differenze_prezzo,media,mediana,%_prezzi_diversi,xor_booking_paga,xor_offerta_solo_mobile,tipo_ricerche,os,username,prezzi_inferiori
21,03-07-2024---11:52 *** 03-07-2024---17:32,405,322,9.791925,9.0,79.506173,0,129,mobile/desktop,Android/Linux,sunnytraveler/marcofantile,Dataframe_2
43,03-07-2024---12:08 *** 03-07-2024---17:32,694,505,11.902970,11.0,72.766571,0,0,desktop/desktop,Windows/Linux,sunnytraveler/marcofantile,Dataframe_2
64,03-07-2024---12:34 *** 03-07-2024---17:32,713,524,11.929389,11.0,73.492286,31,0,desktop/desktop,Linux/Linux,sunnytraveler/marcofantile,Dataframe_2
84,03-07-2024---12:46 *** 03-07-2024---17:32,602,438,9.858447,9.0,72.757475,10,193,mobile/desktop,Android/Linux,sunnytraveler/marcofantile,Dataframe_2
103,03-07-2024---13:02 *** 03-07-2024---17:32,574,406,11.839901,11.0,70.731707,0,0,desktop/desktop,Windows/Linux,sunnytraveler/marcofantile,Dataframe_2
121,03-07-2024---13:14 *** 03-07-2024---17:32,684,500,12.252000,11.0,73.099415,0,0,desktop/desktop,Linux/Linux,sunnytraveler/marcofantile,Dataframe_2
138,03-07-2024---13:25 *** 03-07-2024---17:32,731,535,11.921495,11.0,73.187415,0,0,desktop/desktop,MacOS/Linux,sunnytraveler/marcofantile,Dataframe_2
154,03-07-2024---13:40 *** 03-07-2024---17:32,730,530,12.820755,11.0,72.602740,0,0,desktop/desktop,MacOS/Linux,sunnytraveler/marcofantile,Dataframe_2
169,03-07-2024---14:00 *** 03-07-2024---17:32,634,487,9.554415,9.0,76.813880,0,205,mobile/desktop,Android/Linux,pantilaura56/marcofantile,Dataframe_2
183,03-07-2024---14:20 *** 03-07-2024---17:32,646,495,9.662626,9.0,76.625387,27,215,mobile/desktop,Android/Linux,pantilaura56/marcofantile,Dataframe_2


In [8]:
# Creo grafico a barre per ricerca anomala mobile
# Estraggo dati per grafico a barre dalla ricerca 1
media_diff_prezzo_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['media'].mean()
media_diff_prezzo_desk_mob = zone3r1[zone3r1['tipo_ricerche']!="mobile/mobile"]['media'].mean()

media_num_diff_prezzo_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['num_differenze_prezzo'].mean()
media_xor_booking_paga_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['xor_booking_paga'].mean()
media_xor_offerta_solo_mobile_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['xor_offerta_solo_mobile'].mean()

media_num_diff_prezzo_desk_mob = zone3r1[zone3r1['tipo_ricerche']!="mobile/mobile"]['num_differenze_prezzo'].mean()
media_xor_booking_paga_desk_mob = zone3r1[zone3r1['tipo_ricerche']!="mobile/mobile"]['xor_booking_paga'].mean()
media_xor_offerta_solo_mobile_desk_mob = zone3r1[zone3r1['tipo_ricerche']!="mobile/mobile"]['xor_offerta_solo_mobile'].mean()

print(media_diff_prezzo_desk_mob)


medie_diff_prezzo = [media_num_diff_prezzo_mob_mob, media_num_diff_prezzo_desk_mob]
medie_xor_booking_paga = [media_xor_booking_paga_mob_mob,media_xor_booking_paga_desk_mob]
medie_xor_mobile = [media_xor_offerta_solo_mobile_mob_mob,media_xor_offerta_solo_mobile_desk_mob]

# Definizione delle etichette delle barre
# Definizione della larghezza delle barre
width = 0.25

# Calcolo delle posizioni delle barre
indexs = np.arange(2)
colors = ['lightblue', 'lightgreen']
# Creazione del grafico a barre
plt.style.use("seaborn-v0_8-dark")
plt.bar(indexs, medie_xor_mobile, width=width,color = colors,edgecolor = "grey")
plt.bar(indexs+width, medie_xor_booking_paga, width=width, color =colors,edgecolor = "grey")
plt.bar(indexs+width*2, medie_diff_prezzo, width=width,color = colors,edgecolor = "grey")

# Aggiunta delle etichette
plt.title('Atypical Mobile Search',fontsize=18)
plt.xticks([])
plt.text(-0.125,-25,"Mobile-Only offer", fontsize = 14)
plt.text(0.145,-25,"Booking Pay", fontsize = 14)
plt.text(0.405,-25,"Num Diff Prices", fontsize = 14)
plt.text(-0.125+1.05,-25,"Mobile-Only Offer", fontsize = 14)
plt.text(0.145+1,-25,"Booking Pay", fontsize = 14)
plt.text(0.405+1,-25,"Num Diff Prices", fontsize = 14)
plt.text(0.25,50,"Avg Price Diff\n"+str(media_diff_prezzo_mob_mob.round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(1.25,50,"Avg Price Diff\n"+str(media_diff_prezzo_desk_mob.round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
#plt.xlabel("Per osservare che offerta 'Booking.com paga' è responsabile delle differenze di prezzo")
legend_elements = [
    Patch(facecolor=colors[0], edgecolor='black', label='Mobile vs Mobile'),
    Patch(facecolor=colors[1], edgecolor='black', label='Mobile vs Desktop')
    
]
plt.legend(handles=legend_elements, loc='upper left',fontsize=14)

# Visualizzazione del grafico
plt.show()


17.771535451374522


In [9]:
# Creo grafico per ricerca anomala desktop
media_diff_prezzo_mob_mob = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['media'].mean()
media_diff_prezzo_desk_mob = zone3r2[zone3r2['tipo_ricerche']!="desktop/desktop"]['media'].mean()

media_num_diff_prezzo_mob_mob = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['num_differenze_prezzo'].mean()
media_xor_booking_paga_mob_mob = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['xor_booking_paga'].mean()
media_xor_offerta_solo_mobile_mob_mob = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['xor_offerta_solo_mobile'].mean()

media_num_diff_prezzo_desk_mob = zone3r2[zone3r2['tipo_ricerche']!="desktop/desktop"]['num_differenze_prezzo'].mean()
media_xor_booking_paga_desk_mob = zone3r2[zone3r2['tipo_ricerche']!="desktop/desktop"]['xor_booking_paga'].mean()
media_xor_offerta_solo_mobile_desk_mob = zone3r2[zone3r2['tipo_ricerche']!="desktop/desktop"]['xor_offerta_solo_mobile'].mean()

# Creo grafico a barre per ricerca anomala mobile
medie_diff_prezzo = [media_num_diff_prezzo_mob_mob, media_num_diff_prezzo_desk_mob]
medie_xor_booking_paga = [media_xor_booking_paga_mob_mob,media_xor_booking_paga_desk_mob]
medie_xor_mobile = [media_xor_offerta_solo_mobile_mob_mob,media_xor_offerta_solo_mobile_desk_mob]

# Definizione delle etichette delle barre
# Definizione della larghezza delle barre
width = 0.25

# Calcolo delle posizioni delle barre
indexs = np.arange(2)
colors = ['lightblue', 'lightgreen']
# Creazione del grafico a barre
plt.style.use("seaborn-v0_8-dark")
plt.bar(indexs, medie_xor_mobile, width=width,color = colors,edgecolor = "grey")
plt.bar(indexs+width, medie_xor_booking_paga, width=width, color =colors,edgecolor = "grey")
plt.bar(indexs+width*2, medie_diff_prezzo, width=width,color = colors,edgecolor = "grey")

# Aggiunta delle etichette
plt.title('Atypical Desktop Search',fontsize=18)
plt.xticks([])
plt.text(-0.125,-25,"Mobile-Only Offer", fontsize = 14)
plt.text(0.145,-25,"Booking Pay", fontsize = 14)
plt.text(0.405,-25,"Num Diff Prices", fontsize = 14)
plt.text(-0.125+1.05,-25,"Mobile-Only Offer", fontsize = 14)
plt.text(0.145+1,-25,"Booking Pay", fontsize = 14)
plt.text(0.405+1,-25,"Num Diff Prices", fontsize = 14)
plt.text(0.25,50,"Avg Price Diff\n"+str(media_diff_prezzo_mob_mob.round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(1.25,50,"Avg Price Diff\n"+str(media_diff_prezzo_desk_mob.round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
#plt.xlabel("Per osservare che offerta 'Booking.com paga' è responsabile delle differenze di prezzo")
legend_elements = [
    Patch(facecolor=colors[0], edgecolor='black', label='Desktop vs Desktop'),
    Patch(facecolor=colors[1], edgecolor='black', label='Mobile vs Desktop')
    
]
plt.legend(handles=legend_elements, loc='upper left',fontsize=14)

# Visualizzazione del grafico
plt.show()

In [10]:
media_diff_prezzo_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['media'].mean()

media_num_diff_prezzo_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['num_differenze_prezzo'].mean()
media_xor_booking_paga_mob_mob = zone3r1[zone3r1['tipo_ricerche']=="mobile/mobile"]['xor_booking_paga'].mean()


media_diff_prezzo_desk_desk = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['media'].mean()

media_num_diff_prezzo_desk_desk = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['num_differenze_prezzo'].mean()
media_xor_booking_paga_desk_desk = zone3r2[zone3r2['tipo_ricerche']=="desktop/desktop"]['xor_booking_paga'].mean()

medie_diff_prezzo = [media_num_diff_prezzo_mob_mob, media_num_diff_prezzo_desk_desk]
medie_xor_booking_paga = [media_xor_booking_paga_mob_mob,media_xor_booking_paga_desk_desk]


width = 0.25

# Calcolo delle posizioni delle barre
indexs = np.arange(2)
colors = ['lightblue', 'lightgreen']
# Creazione del grafico a barre
plt.style.use("seaborn-v0_8-dark")

plt.bar(indexs+width, medie_xor_booking_paga, width=width, color =colors,edgecolor = "grey")
plt.bar(indexs+width*2, medie_diff_prezzo, width=width,color = colors,edgecolor = "grey")

# Aggiunta delle etichette
plt.title('Atypical Search',fontsize=18)
plt.xticks([])

plt.text(0.145,-25,"Booking Pay", fontsize = 14)
plt.text(0.405,-25,"Num Diff Prices", fontsize = 14)

plt.text(0.145+1,-25,"Booking Pay", fontsize = 14)
plt.text(0.405+1,-25,"Num Diff Prices", fontsize = 14)
plt.text(0.25,50,"Avg Price Diff\n"+str(media_diff_prezzo_mob_mob.round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(1.25,50,"Avg Price Diff\n"+str(media_diff_prezzo_desk_desk.round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
#plt.xlabel("Per osservare che offerta 'Booking.com paga' è responsabile delle differenze di prezzo")
legend_elements = [
    Patch(facecolor=colors[0], edgecolor='black', label='Mobile vs Mobile'),
    Patch(facecolor=colors[1], edgecolor='black', label='Desktop vs Desktop')
    
]
plt.legend(handles=legend_elements, loc='upper center',fontsize=14,)

# Visualizzazione del grafico
plt.show()

In [11]:
# Creo Grafico per zona 1 e 2 
plt.style.use("seaborn-v0_8-dark")

medie_diff_prezzo = [zone1['num_differenze_prezzo'].mean(), zone2['num_differenze_prezzo'].mean()]
medie_xor_mobile = [zone1['xor_offerta_solo_mobile'].mean(),zone2['xor_offerta_solo_mobile'].mean()]
medie_xor_booking_paga = [zone1['xor_booking_paga'].mean(),zone2['xor_booking_paga'].mean()]

indexs = np.arange(2)
colors = ['lightblue', 'lightgreen']
width=0.25

plt.bar(indexs, medie_xor_mobile, width=width,color = colors,edgecolor = "grey")
plt.bar(indexs+width, medie_xor_booking_paga, width=width, color =colors,edgecolor = "grey")
plt.bar(indexs+width*2, medie_diff_prezzo, width=width,color = colors,edgecolor = "grey")

# Aggiunta delle etichette
plt.title('Typical Search',fontsize=18)
plt.xticks([])
plt.text(-0.125,-25,"Mobile-Only Offer", fontsize = 14)
plt.text(0.145,-25,"Booking Pay", fontsize = 14)
plt.text(0.405,-25,"Num Diff Prices", fontsize = 14)
plt.text(-0.125+1.05,-25,"Mobile-Only Offer", fontsize = 14)
plt.text(0.145+1,-25,"Booking Pay", fontsize = 14)
plt.text(0.405+1,-25,"Num Diff Prices", fontsize = 14)
plt.text(0.25,50,"Avg Price Diff\n"+str(zone1['media'].mean().round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(1.25,50,"Avg Price Diff\n"+str(zone2['media'].mean().round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
#plt.xlabel("Per osservare che offerta 'Booking.com paga' è responsabile delle differenze di prezzo")
legend_elements = [
    Patch(facecolor=colors[0], edgecolor='black', label='Desktop vs Desktop or Mobile vs Mobile'),
    Patch(facecolor=colors[1], edgecolor='black', label='Mobile vs Desktop')
    
]
plt.legend(handles=legend_elements, loc='upper left',fontsize=14)

# Visualizzazione del grafico
plt.show()

In [12]:
# Elimino una o più ricerche dal dataframe
new_df = df_diff_prezzo.drop(df_diff_prezzo[df_diff_prezzo['coppia_ricerche'].str.contains("20:42")].index) 
new_df.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

,coppia_ricerche,totale_corrispondenze,num_differenze_prezzo,media,mediana,%_prezzi_diversi,xor_booking_paga,xor_offerta_solo_mobile,tipo_ricerche,os,username,prezzi_inferiori
179,03-07-2024---14:20 *** 03-07-2024---16:09,936,697,11.609756,10.0,74.465812,659,0,mobile/mobile,Android/Android,pantilaura56/marcofantile,Dataframe_2
165,03-07-2024---14:00 *** 03-07-2024---16:09,909,676,11.857988,10.0,74.367437,675,0,mobile/mobile,Android/Android,pantilaura56/marcofantile,Dataframe_2
275,03-07-2024---17:32 *** 03-07-2024---17:51,900,656,11.592988,11.0,72.888889,0,0,desktop/desktop,Linux/MacOS,marcofantile/marcofantile,Dataframe_1
273,03-07-2024---17:21 *** 03-07-2024---17:32,889,644,11.740683,11.0,72.440945,34,0,desktop/desktop,Windows/Linux,marcofantile/marcofantile,Dataframe_2
234,03-07-2024---15:20 *** 03-07-2024---16:09,801,622,17.948553,14.0,77.652934,537,271,desktop/mobile,Linux/Android,pantilaura56/marcofantile,Dataframe_2
...,...,...,...,...,...,...,...,...,...,...,...,...
230,03-07-2024---15:08 *** 03-07-2024---17:51,759,0,NaN,NaN,0.000000,0,0,desktop/desktop,Linux/MacOS,pantilaura56/marcofantile,Dataframe_2
252,03-07-2024---15:43 *** 03-07-2024---17:21,762,0,NaN,NaN,0.000000,0,0,desktop/desktop,MacOS/Windows,pantilaura56/marcofantile,Dataframe_2
270,03-07-2024---17:03 *** 03-07-2024---17:21,873,0,NaN,NaN,0.000000,0,0,desktop/desktop,Windows/Windows,marcofantile/marcofantile,Dataframe_2
272,03-07-2024---17:03 *** 03-07-2024---17:51,874,0,NaN,NaN,0.000000,0,0,desktop/desktop,Windows/MacOS,marcofantile/marcofantile,Dataframe_2


In [13]:
# Visualizza il numero di ricerche mobile e desktop
mobile = 0
desktop = 0
for i in df_list:
    df1 = df[df["TIMESTAMP"]==i]
    stringa = df1['user_agent_type'].unique()[0]
    if "mob" in stringa:
        mobile +=1
    else: desktop+=1
    print(df1['user_agent_type'].unique())

print("Mobile:", mobile,"Desktop:",desktop)

['mobile']
['desktop']
['desktop']
['mobile']
['desktop']
['desktop']
['desktop']
['desktop']
['mobile']
['mobile']
['desktop']
['desktop']
['desktop']
['desktop']
['desktop']
['desktop']
['desktop']
['mobile']
['mobile']
['desktop']
['desktop']
['desktop']
['desktop']
['desktop']
Mobile: 6 Desktop: 18


In [14]:
# Ora faccio Merge per confrontare prezzi tra OS
df['os'].unique()

os1 = 'MacOS'
os2 = 'Windows'
search = "17:32"
selected_os_list = df[(df['os']==os1) & (~df['TIMESTAMP'].str.contains(search))]['TIMESTAMP'].unique()
other_os_list = df[(df['os']==os2) & (~df['TIMESTAMP'].str.contains(search))]['TIMESTAMP'].unique()


os_pairs = [(x,y) for x in selected_os_list for y in other_os_list]
list_dict_df = []

for i in os_pairs:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    #merged_solo_nome = pd.merge(df1,df2, on='nome_hotel')
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"])
    media_positivi = merged_df.loc[merged_df['differenza_prezzo'] > 0, 'differenza_prezzo'].mean()
    media_negativi = merged_df.loc[merged_df['differenza_prezzo'] < 0, 'differenza_prezzo'].mean()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe_1"
    else: prezzi_inferiori = "Dataframe_2"
    try: 
        xor_booking_paga = len(merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])])
    except:
        xor_booking_paga = None
    
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df['totale_corrispondenze'] = initial_len
    dict_df ['num_diff_prezzo'] = len(merged_df)
    dict_df ['num_diff_prezzo_negative'] = len(merged_df[merged_df['prezzo_x']<merged_df['prezzo_y']])
    dict_df ['num_diff_prezzo_positive'] = len(merged_df[merged_df['prezzo_x']>merged_df['prezzo_y']])
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['media_positivi'] = media_positivi
    dict_df ['media_negativi'] = media_negativi
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len*100
    #dict_df['hotel_scartati'] = len(merged_solo_nome) - initial_len
    dict_df['xor_booking_paga'] = xor_booking_paga
    dict_df['os'] = df1['os'].unique()[0] + "/"+ df2['os'].unique()[0]
    dict_df['username'] = df1['username'].unique()[0] + "/"+ df2['username'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    list_dict_df.append(dict_df)

df_diff_prezzo = pd.DataFrame(list_dict_df)   
#pd.set_option('display.max_rows', None)

df_diff_prezzo = df_diff_prezzo[df_diff_prezzo['media'].notnull()]
df_diff_prezzo.sort_values(by=['num_diff_prezzo'],ascending= False)


,coppia_ricerche,totale_corrispondenze,num_diff_prezzo,num_diff_prezzo_negative,num_diff_prezzo_positive,media,media_positivi,media_negativi,mediana,%_prezzi_diversi,xor_booking_paga,os,username,prezzi_inferiori
1,03-07-2024---13:25 *** 03-07-2024---13:02,750,42,41,1,-22.404762,18.000000,-23.390244,-19.5,5.600000,0,MacOS/Windows,sunnytraveler/sunnytraveler,Dataframe_1
13,03-07-2024---15:32 *** 03-07-2024---13:02,721,40,39,1,-22.625000,18.000000,-23.666667,-19.5,5.547850,39,MacOS/Windows,pantilaura56/sunnytraveler,Dataframe_1
6,03-07-2024---13:40 *** 03-07-2024---12:08,887,37,0,37,23.621622,23.621622,NaN,20.0,4.171364,0,MacOS/Windows,sunnytraveler/sunnytraveler,Dataframe_2
8,03-07-2024---13:40 *** 03-07-2024---14:38,920,37,0,37,24.081081,24.081081,NaN,21.0,4.021739,37,MacOS/Windows,sunnytraveler/pantilaura56,Dataframe_2
10,03-07-2024---13:40 *** 03-07-2024---17:03,778,34,0,34,23.235294,23.235294,NaN,22.5,4.370180,34,MacOS/Windows,sunnytraveler/marcofantile,Dataframe_2
31,03-07-2024---17:51 *** 03-07-2024---13:02,592,29,29,0,-26.310345,NaN,-26.310345,-23.0,4.898649,0,MacOS/Windows,marcofantile/sunnytraveler,Dataframe_1
11,03-07-2024---13:40 *** 03-07-2024---17:21,739,25,0,25,24.440000,24.440000,NaN,22.0,3.382950,25,MacOS/Windows,sunnytraveler/marcofantile,Dataframe_2
25,03-07-2024---16:51 *** 03-07-2024---13:02,709,25,10,15,11.000000,27.800000,-14.200000,7.0,3.526093,0,MacOS/Windows,marcofantile/sunnytraveler,Dataframe_2
28,03-07-2024---16:51 *** 03-07-2024---17:03,798,23,0,23,27.043478,27.043478,NaN,17.0,2.882206,1,MacOS/Windows,marcofantile/marcofantile,Dataframe_2
27,03-07-2024---16:51 *** 03-07-2024---14:49,891,23,0,23,29.565217,29.565217,NaN,19.0,2.581369,1,MacOS/Windows,marcofantile/pantilaura56,Dataframe_2


In [15]:
# Quando le differenze di prezzo sono negative abbiamo costì più bassi su MacOS
num_prezzi_bassi_mac = df_diff_prezzo['num_diff_prezzo_negative'].sum()
num_prezzi_bassi_win = df_diff_prezzo['num_diff_prezzo_positive'].sum()

# Media Pesata
media_pesata_tot= ((df_diff_prezzo['num_diff_prezzo']*df_diff_prezzo['media']).sum())/(df_diff_prezzo['num_diff_prezzo'].sum())

media_pesata_negativi = ((df_diff_prezzo['num_diff_prezzo_negative']*df_diff_prezzo['media_negativi']).sum())/(df_diff_prezzo['num_diff_prezzo_negative'].sum())
media_pesata_positivi = ((df_diff_prezzo['num_diff_prezzo_positive']*df_diff_prezzo['media_positivi']).sum())/(df_diff_prezzo['num_diff_prezzo_positive'].sum())
num_prezzi_bassi_mac



129

In [16]:
# Ora faccio Merge per confrontare prezzi tra OS
import random 
df['os'].unique()

os1 = 'MacOS'
os2 = 'Linux'
search = "17:32"
selected_os_list = df[(df['os']==os1) & (~df['TIMESTAMP'].str.contains(search))]['TIMESTAMP'].unique()
elemento_random = np.random.randint(0,len(selected_os_list))
selected_os_list = np.delete(selected_os_list,elemento_random)
other_os_list = df[(df['os']==os2) & (~df['TIMESTAMP'].str.contains(search))]['TIMESTAMP'].unique()


os_pairs = [(x,y) for x in selected_os_list for y in other_os_list]
list_dict_df = []

for i in os_pairs:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    #merged_solo_nome = pd.merge(df1,df2, on='nome_hotel')
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"])
    media_positivi = merged_df.loc[merged_df['differenza_prezzo'] > 0, 'differenza_prezzo'].mean()
    media_negativi = merged_df.loc[merged_df['differenza_prezzo'] < 0, 'differenza_prezzo'].mean()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe_1"
    else: prezzi_inferiori = "Dataframe_2"
    try: 
        xor_booking_paga = len(merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])])
    except:
        xor_booking_paga = None
    
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df['totale_corrispondenze'] = initial_len
    dict_df ['num_diff_prezzo'] = len(merged_df)
    dict_df ['num_diff_prezzo_negative'] = len(merged_df[merged_df['prezzo_x']<merged_df['prezzo_y']])
    dict_df ['num_diff_prezzo_positive'] = len(merged_df[merged_df['prezzo_x']>merged_df['prezzo_y']])
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['media_positivi'] = media_positivi
    dict_df ['media_negativi'] = media_negativi
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len*100
    #dict_df['hotel_scartati'] = len(merged_solo_nome) - initial_len
    dict_df['xor_booking_paga'] = xor_booking_paga
    dict_df['os'] = df1['os'].unique()[0] + "/"+ df2['os'].unique()[0]
    dict_df['username'] = df1['username'].unique()[0] + "/"+ df2['username'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    list_dict_df.append(dict_df)

df_diff_prezzo = pd.DataFrame(list_dict_df)   
#pd.set_option('display.max_rows', None)

df_diff_prezzo = df_diff_prezzo[df_diff_prezzo['media'].notnull()]
df_diff_prezzo.sort_values(by=['num_diff_prezzo'],ascending= False)


,coppia_ricerche,totale_corrispondenze,num_diff_prezzo,num_diff_prezzo_negative,num_diff_prezzo_positive,media,media_positivi,media_negativi,mediana,%_prezzi_diversi,xor_booking_paga,os,username,prezzi_inferiori
9,03-07-2024---13:40 *** 03-07-2024---15:20,926,56,17,39,10.107143,24.128205,-22.058824,16.0,6.047516,39,MacOS/Linux,sunnytraveler/pantilaura56,Dataframe_2
2,03-07-2024---13:25 *** 03-07-2024---14:59,940,43,42,1,-4.093023,18.000000,-4.619048,-4.0,4.574468,42,MacOS/Linux,sunnytraveler/marcofantile,Dataframe_1
7,03-07-2024---13:40 *** 03-07-2024---14:59,937,40,0,40,19.375000,19.375000,NaN,17.5,4.268943,40,MacOS/Linux,sunnytraveler/marcofantile,Dataframe_2
5,03-07-2024---13:40 *** 03-07-2024---12:34,926,39,0,39,23.717949,23.717949,NaN,20.0,4.211663,39,MacOS/Linux,sunnytraveler/sunnytraveler,Dataframe_2
8,03-07-2024---13:40 *** 03-07-2024---15:08,887,39,0,39,23.358974,23.358974,NaN,20.0,4.396843,39,MacOS/Linux,sunnytraveler/pantilaura56,Dataframe_2
22,03-07-2024---17:51 *** 03-07-2024---14:59,779,34,34,0,-4.970588,NaN,-4.970588,-4.0,4.364570,34,MacOS/Linux,marcofantile/marcofantile,Dataframe_1
19,03-07-2024---16:51 *** 03-07-2024---15:20,871,32,13,19,9.937500,34.210526,-25.538462,9.5,3.673938,1,MacOS/Linux,marcofantile/pantilaura56,Dataframe_2
17,03-07-2024---16:51 *** 03-07-2024---14:59,870,31,9,22,20.806452,30.545455,-3.000000,11.0,3.563218,10,MacOS/Linux,marcofantile/marcofantile,Dataframe_2
18,03-07-2024---16:51 *** 03-07-2024---15:08,859,23,0,23,30.565217,30.565217,NaN,21.0,2.677532,1,MacOS/Linux,marcofantile/pantilaura56,Dataframe_2
16,03-07-2024---16:51 *** 03-07-2024---13:14,846,21,0,21,32.095238,32.095238,NaN,22.0,2.482270,0,MacOS/Linux,marcofantile/sunnytraveler,Dataframe_2


In [17]:
# Quando le differenze di prezzo sono negative abbiamo costì più bassi su MacOS
num_prezzi_bassi_mac2 = df_diff_prezzo['num_diff_prezzo_negative'].sum()
num_prezzi_bassi_lin2 = df_diff_prezzo['num_diff_prezzo_positive'].sum()

# Media Pesata
media_pesata_tot2= ((df_diff_prezzo['num_diff_prezzo']*df_diff_prezzo['media']).sum())/(df_diff_prezzo['num_diff_prezzo'].sum())

media_pesata_negativi2 = ((df_diff_prezzo['num_diff_prezzo_negative']*df_diff_prezzo['media_negativi']).sum())/(df_diff_prezzo['num_diff_prezzo_negative'].sum())
media_pesata_positivi2 = ((df_diff_prezzo['num_diff_prezzo_positive']*df_diff_prezzo['media_positivi']).sum())/(df_diff_prezzo['num_diff_prezzo_positive'].sum())

media_pesata_positivi2



25.779783393501805

In [18]:
# Ora faccio Merge per confrontare prezzi tra OS
import random 
df['os'].unique()

os1 = 'Windows'
os2 = 'Linux'
search = "17:32"
selected_os_list = df[(df['os']==os1) & (~df['TIMESTAMP'].str.contains(search))]['TIMESTAMP'].unique()
elemento_random = np.random.randint(0,len(selected_os_list))
selected_os_list = np.delete(selected_os_list,elemento_random)
other_os_list = df[(df['os']==os2) & (~df['TIMESTAMP'].str.contains(search))]['TIMESTAMP'].unique()


os_pairs = [(x,y) for x in selected_os_list for y in other_os_list]
list_dict_df = []

for i in os_pairs:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    #merged_solo_nome = pd.merge(df1,df2, on='nome_hotel')
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"])
    media_positivi = merged_df.loc[merged_df['differenza_prezzo'] > 0, 'differenza_prezzo'].mean()
    media_negativi = merged_df.loc[merged_df['differenza_prezzo'] < 0, 'differenza_prezzo'].mean()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe_1"
    else: prezzi_inferiori = "Dataframe_2"
    try: 
        xor_booking_paga = len(merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])])
    except:
        xor_booking_paga = None
    
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df['totale_corrispondenze'] = initial_len
    dict_df ['num_diff_prezzo'] = len(merged_df)
    dict_df ['num_diff_prezzo_negative'] = len(merged_df[merged_df['prezzo_x']<merged_df['prezzo_y']])
    dict_df ['num_diff_prezzo_positive'] = len(merged_df[merged_df['prezzo_x']>merged_df['prezzo_y']])
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['media_positivi'] = media_positivi
    dict_df ['media_negativi'] = media_negativi
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len*100
    #dict_df['hotel_scartati'] = len(merged_solo_nome) - initial_len
    dict_df['xor_booking_paga'] = xor_booking_paga
    dict_df['os'] = df1['os'].unique()[0] + "/"+ df2['os'].unique()[0]
    dict_df['username'] = df1['username'].unique()[0] + "/"+ df2['username'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    list_dict_df.append(dict_df)

df_diff_prezzo = pd.DataFrame(list_dict_df)   
#pd.set_option('display.max_rows', None)

df_diff_prezzo = df_diff_prezzo[df_diff_prezzo['media'].notnull()]
df_diff_prezzo.sort_values(by=['num_diff_prezzo'],ascending= False)


,coppia_ricerche,totale_corrispondenze,num_diff_prezzo,num_diff_prezzo_negative,num_diff_prezzo_positive,media,media_positivi,media_negativi,mediana,%_prezzi_diversi,xor_booking_paga,os,username,prezzi_inferiori
9,03-07-2024---13:02 *** 03-07-2024---15:20,729,51,13,38,12.352941,23.921053,-21.461538,16.0,6.995885,38,Windows/Linux,sunnytraveler/pantilaura56,Dataframe_2
5,03-07-2024---13:02 *** 03-07-2024---12:34,749,40,0,40,23.675000,23.675000,NaN,20.5,5.340454,40,Windows/Linux,sunnytraveler/sunnytraveler,Dataframe_2
7,03-07-2024---13:02 *** 03-07-2024---14:59,732,39,0,39,19.205128,19.205128,NaN,17.0,5.327869,39,Windows/Linux,sunnytraveler/marcofantile,Dataframe_2
12,03-07-2024---14:38 *** 03-07-2024---14:59,930,39,39,0,-4.615385,NaN,-4.615385,-4.0,4.193548,0,Windows/Linux,pantilaura56/marcofantile,Dataframe_1
2,03-07-2024---12:08 *** 03-07-2024---14:59,879,37,37,0,-4.648649,NaN,-4.648649,-4.0,4.209329,37,Windows/Linux,sunnytraveler/marcofantile,Dataframe_1
8,03-07-2024---13:02 *** 03-07-2024---15:08,724,37,0,37,23.351351,23.351351,NaN,20.0,5.110497,37,Windows/Linux,sunnytraveler/pantilaura56,Dataframe_2
22,03-07-2024---17:21 *** 03-07-2024---14:59,773,27,27,0,-4.740741,NaN,-4.740741,-4.0,3.492885,0,Windows/Linux,marcofantile/marcofantile,Dataframe_1
14,03-07-2024---14:38 *** 03-07-2024---15:20,915,18,18,0,-21.500000,NaN,-21.500000,-17.5,1.967213,0,Windows/Linux,pantilaura56/pantilaura56,Dataframe_1
4,03-07-2024---12:08 *** 03-07-2024---15:20,872,16,16,0,-22.375000,NaN,-22.375000,-18.0,1.834862,0,Windows/Linux,sunnytraveler/pantilaura56,Dataframe_1
24,03-07-2024---17:21 *** 03-07-2024---15:20,771,15,15,0,-21.266667,NaN,-21.266667,-16.0,1.945525,0,Windows/Linux,marcofantile/pantilaura56,Dataframe_1


In [19]:
# Quando le differenze di prezzo sono negative abbiamo costì più bassi su MacOS
num_prezzi_bassi_win3 = df_diff_prezzo['num_diff_prezzo_negative'].sum()
num_prezzi_bassi_lin3 = df_diff_prezzo['num_diff_prezzo_positive'].sum()

# Media Pesata
media_pesata_tot3= ((df_diff_prezzo['num_diff_prezzo']*df_diff_prezzo['media']).sum())/(df_diff_prezzo['num_diff_prezzo'].sum())

media_pesata_negativi3 = ((df_diff_prezzo['num_diff_prezzo_negative']*df_diff_prezzo['media_negativi']).sum())/(df_diff_prezzo['num_diff_prezzo_negative'].sum())
media_pesata_positivi3 = ((df_diff_prezzo['num_diff_prezzo_positive']*df_diff_prezzo['media_positivi']).sum())/(df_diff_prezzo['num_diff_prezzo_positive'].sum())

num_prezzi_bassi_win3
media_pesata_tot3
#num_prezzi_bassi_win3


4.101983002832861

In [22]:
num_prezzi_bassi_primo = [num_prezzi_bassi_mac,num_prezzi_bassi_mac2,num_prezzi_bassi_win3]
num_prezzi_bassi_secondo = [num_prezzi_bassi_win,num_prezzi_bassi_lin2,num_prezzi_bassi_lin3]

# Definizione delle etichette delle barre
# Definizione della larghezza delle barre
width = 0.25

# Calcolo delle posizioni delle barre
indexs = np.arange(3)
colors = ['lightblue', 'lightblue','#D81E5B']
colors2 = ['#D81E5B', 'lightgreen','lightgreen']

# Creazione del grafico a barre
plt.style.use("seaborn-v0_8-dark")
plt.title("Number of Different Prices by OS",fontsize=18)
plt.bar(indexs, num_prezzi_bassi_primo , width=width,color=colors,edgecolor = "grey")
plt.bar(indexs+width, num_prezzi_bassi_secondo, width=width,color = colors2,edgecolor = "grey")

legend_elements = [
    Patch(facecolor=colors[0], edgecolor='black', label='MacOS'),
    Patch(facecolor=colors[2], edgecolor='black', label='Windows'),
    Patch(facecolor= colors2[2], edgecolor='black', label='Linux')
    
]
plt.legend(handles=legend_elements, loc='upper left',fontsize=14)
plt.text(0.02,15,str(abs(media_pesata_negativi).round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(0.26,15,str(abs(media_pesata_positivi).round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(1,15,str(abs(media_pesata_negativi2).round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(1.26,15,str(abs(media_pesata_positivi2).round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(2,15,str(abs(media_pesata_negativi3).round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.text(2.26,15,str(abs(media_pesata_positivi3).round(2))+'€', fontsize=14, color='black', ha='center',fontweight = "bold")
plt.xticks([])


plt.show()


In [23]:
len()

25